## Laboratorio 07

- Sara Echeverría 
- Ricardo Méndez   
- Melissa Pérez 

Repositorio: https://github.com/MelissaPerez09/Lab07-CC3104

### Task 01
**Explique la diferencia entre los métodos de aprendizaje de refuerzo basados en valores y en políticas.**
- Los métodos basados en valores buscan aprender una función que estime qué tan buena es una acción en un estado y luego eligen la mejor acción comparando esos valores. A partir de ellas derivan una política que elige las mejores acciones.
- Los métodos basados en políticas aprenden directamente de una política parametrizada que indica cómo actuar en cada estado sin necesidad de comparar entre todas las posibles. 

**¿Por qué los métodos de gradiente de políticas son especialmente útiles para entornos con espacios de acción continua?**
- Porque permiten generar acciones optimizadas en el comportamiento sin tener la necesidad de evaluar y compara un número de acciones. 